<a href="https://colab.research.google.com/github/ghubesharman/DataSciencePython/blob/main/Cozie_Face_Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Install Required Libraries**
Install dlib, opencv-python, and scipy if they’re not already installed.


In [25]:
!pip install dlib opencv-python-headless scipy

# **2. Download and Decompress Pre-trained Models**
This cell will download and decompress the necessary Dlib models.



In [26]:
import urllib.request
import bz2
import os
import cv2
import dlib
import numpy as np
from scipy.spatial.distance import cosine

# Define URLs and paths for the pre-trained models
landmark_model_url = "http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2"
face_rec_model_url = "http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2"
landmark_model_path = "/content/shape_predictor_5_face_landmarks.dat"
face_rec_model_path = "/content/dlib_face_recognition_resnet_model_v1.dat"

# Function to download and decompress models
def download_and_decompress(url, output_path):
    compressed_path = output_path + ".bz2"
    if not os.path.exists(output_path):
        urllib.request.urlretrieve(url, compressed_path)
        with bz2.BZ2File(compressed_path, "rb") as compressed_file:
            with open(output_path, "wb") as decompressed_file:
                decompressed_file.write(compressed_file.read())
        print(f"{output_path} downloaded and decompressed.")
    else:
        print(f"{output_path} already exists. Skipping download.")

# Download and load models
download_and_decompress(landmark_model_url, landmark_model_path)
download_and_decompress(face_rec_model_url, face_rec_model_path)

face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(landmark_model_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)


/content/shape_predictor_5_face_landmarks.dat already exists. Skipping download.
/content/dlib_face_recognition_resnet_model_v1.dat already exists. Skipping download.


# **3. Define Helper Functions**
Combine detect_face, generate_embedding, and verify_face functions in a single cell to keep all essential logic together.

In [27]:
def detect_face(image):
    """
    Detects faces in an image and returns aligned face regions.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    face_regions = []
    for face in faces:
        landmarks = shape_predictor(image, face)
        aligned_face = dlib.get_face_chip(image, landmarks, size=150)
        face_regions.append(aligned_face)
    return face_regions

def generate_embedding(face):
    """
    Generates a 128-dimensional embedding for the face using Dlib's ResNet model.
    """
    face_embedding = np.array(face_rec_model.compute_face_descriptor(face))
    return face_embedding

def verify_face(new_embedding, reference_embeddings, threshold=0.6):
    """
    Compares a new face embedding with reference embeddings to verify identity.
    """
    for ref_embedding in reference_embeddings:
        similarity = 1 - cosine(new_embedding, ref_embedding)
        print(f"Similarity with reference: {similarity:.2f}")
        if similarity >= threshold:
            return True
    return False


# **4. Upload and Generate Reference Embeddings**
This cell handles uploading multiple reference images and generating embeddings for them, all in one place.

In [32]:
# Upload reference images for a user (up to 6 images)
from google.colab import files
uploaded_refs = files.upload()  # Upload up to 6 reference images

# Generate embeddings for each uploaded reference image
reference_embeddings = []
for ref_image_path in uploaded_refs.keys():
    image = cv2.imread(ref_image_path)
    faces = detect_face(image)
    if faces:
        ref_embedding = generate_embedding(faces[0])
        reference_embeddings.append(ref_embedding)
    else:
        print(f"No face detected in {ref_image_path}")

print("Reference embeddings generated and stored.")


Saving Gym.jpg to Gym (9).jpg
Saving me nd antara.jpg to me nd antara (1).jpg
Saving Me.jpg to Me (7).jpg
Saving Yakii.jpg to Yakii (7).jpg
Reference embeddings generated and stored.


# **5. Upload a New Image for Verification**
This cell handles uploading a new image, generating an embedding, and performing the verification.

In [33]:
# Verify that reference_embeddings exists
if 'reference_embeddings' not in globals() or not reference_embeddings:
    print("Error: 'reference_embeddings' is not defined or is empty. Please run the cell that generates reference embeddings first.")

# Upload a new image for face verification
uploaded_new = files.upload()  # Upload a single new image for verification

# Load the uploaded image for verification
new_image_path = list(uploaded_new.keys())[0]
image = cv2.imread(new_image_path)

# Detect face and generate embedding for the new image
faces = detect_face(image)
if faces:
    new_embedding = generate_embedding(faces[0])

    # Perform verification against the reference embeddings
    result = verify_face(new_embedding, reference_embeddings)
    if result:
        print("Face verification successful!")
    else:
        print("Face verification failed.")
else:
    print("No faces detected in the new image.")


Saving test.jpg to test.jpg
Similarity with reference: 0.95
Face verification successful!
